In [ ]:
import time
import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import networkx as nx
import scipy
import pandas as pd
from scipy.optimize import curve_fit

**WOLF AND KANTZ ALGORITHM FOR TEMPORAL NETWORK**

This code uses the Wolf or Kantz algorithm to analyze temporal networks and measure their unpredictability. Specifically, the algorithm returns the distribution of local expansion rates and estimates the maximum Lyapunov exponent.

**Distance**

In the next cell we define the distance between networks.
As an example here we use the distance between their adjacency matrices.

In [ ]:
def distance(x,y):
    N=len(x) 
    d=np.sum(abs(x-y)/(N*(N-1))) #difference between adjacency matrix
    return d

**Goodness of fit: R²**

Moreover, for each value of the local expansion rate, we obtain a fit and an R-squared value. The wolf algorithm, which we will introduce later, will filter out values of local expansion rate with an R-squared value less than 0.9.

In [ ]:
def Rsquared(x,y,y_tot,f): # compute the R-squared
    z=np.linspace(0,len(y_tot)-1,len(y_tot))
    y_bar = np.mean(y)
    ss_tot = ((y-y_bar)**2).sum()
    ss_res = ((y-f)**2).sum()
    return 1 - (ss_res/ss_tot)

def best_fit(y_nl,window): # perform the fit over a window of points 
    y=np.log(np.array(y_nl)) 
    best_R=0
    i=0
    x=np.linspace(0,len(y)-1,len(y)) 
    a,b=np.polyfit(x[i:i+window],y[i:i+window],1)
    f=a*x+b
    a_old=a
    R=Rsquared(x[i:i+window], y[i:i+window],y,f[i:i+window])
    best_R=R
    while  i<len(y)-window and R<0.9:
        a_old=a
        best_R=R
        i=i+1
        a,b=np.polyfit(x[i:i+window],y[i:i+window],1)
        f=a*x+b
        R=Rsquared(x[i:i+window], y[i:i+window],y,f[i:i+window])
        
    return a_old,best_R,i-1

**Description**

The algorithms require the following parameters in this order:

* A time series of networks represented by the adjacency matrix.
* The total time length of the trajectory. Generally, it is the length of the time series, but one can also consider a part of it.
* The initial point of the trajectory.
* The parameter $\tau$ (saturation time for the dynamical system).
* $w$, the number of points on which we compute the fit.
* (Only for Kantz) the radius of the sphere $\epsilon$.

It returns:

* An array of unknown length of local expansion rates.
* The estimated Maximum Lyapunov exponent.

In [ ]:

def Wolf(Trajectory, Time, transient,tau,w):

    i=transient   # Index for referring to the initial point
     
    dista=[] # Array for distances 
    
    S=[] # Initialize the list of local expansion rates
    
    while i< Time-tau-1 : # check to do not exceed the time limit
    
        xi=Trajectory[i] # we take the value of the evolution of the initial point at time transient+ tau*iteration
       
        d_old=distance(xi,Trajectory[transient+1])+1 # Initialize the first distance

        for j in range(transient+1,Time-tau-1): #compute the minimum distance between xi and another point in the trajectory
            d_new=distance(xi,Trajectory[j])
            d_new_j=j
            if d_new > 0:
                if d_new< d_old:
                    d_old=d_new
                    d_old_j=d_new_j
    
        xk=Trajectory[d_old_j] # the point associated tot the minimum distance
        dista=[] 
        z=0
        dista.append(distance(xk,xi)) 
        while  z<tau-1: # Advance until tau
            z=z+1 # advancing in time
            ds=distance(xk,xi) # save previous distance
            xi=Trajectory[i+z] # compute the evolution of xi
            xk=Trajectory[d_old_j+z] # compute the evolution of xk
            dista.append(distance(xk,xi)) #append the distance at time z
            
        s,r,inp=best_fit(dista,w) # compute the fit on the array of distances
        if r>=0.9: # select value with R-squared >=0.9
            S.append(s)
           

        i=i+tau  #the initial point advancing in time 
        
    return S, np.mean(S)

      
        

In [ ]:

def Kantz(Trajectory, Time, transient,tau,w,eps):
  
    i=transient   # Index for referring to the initial point
     
    dista=[] # Initialize array for distances 
    
    S=[] # Initialize the list of local expansion rates
    
    while i<Time-tau-1: # check to do not exceed the time limit
        
        xi=Trajectory[i] # we take the value of the evolution of the initial point at time transient+tau*iteration
       
        # Initialize some arrays
        dist=[]
        dista=[]
        distj=[]
        
        for j in range(i,Time-tau-1): #compute all the distances of xi with other points in the series, these distances must be greater than 0 and lower than epsilon
            if distance(xi,Trajectory[j])>0. and distance(xi,Trajectory[j])<eps:

                dist.append(distance(xi,Trajectory[j]))
                distj.append(j)
        
        if len(dist)==0:
            print('no points in the ball')
            break

        d0=np.array(dist) # save the array of distances
      
        z=0
   
        dista.append(np.mean(d0)) # save the first mean value of the distances

        while  z<tau-1: # Advance until tau
            z=z+1 # Advancing in time
            ds=np.array(dist) # save previous array for distances
            
            for d in range(len(dist)): # compute the evolution of the distance of the points in the ball
                dist[d]=distance(Trajectory[i+z],Trajectory[distj[d]+z])
                
            dista.append(np.mean(dist))
            
        s,r,inp=best_fit(dista,w) # compute the fit on the array of distances
        if r>=0.9: # select value with R-squared >=0.9
            S.append(s)
           

        i=i+tau  #the initial point advancing in time 
        
        
    return S, np.mean(S)
      
